In [1]:
!pip install fake_useragent

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


В этой функции вытаскиваем ссылки на фильмы, чтобы потом смочь из каждого фильма выкачать отзывы на него.

In [3]:
def get_films(year):
  links = []

  year = str(year)
  response = session.get(f"https://www.metacritic.com/browse/movies/score/metascore/year/filtered/netflix?year_selected={year}&sort=desc&view=detailed", headers={'User-Agent': ua.random})
  soup = BeautifulSoup(response.text, "html.parser")
  films = soup.find_all('tr')
  for film in films:
    link = film.find('a', {'class': 'title'})
    if link is not None:
      link = link.attrs['href']
      links.append(link)
  return links

В этой длинной (простите, короче не вышло) функции вытаскиваем отзывы на определенный фильм. И сразу же распихиваем, положительный или отрицательный отзыв. Для простоты я решила не брать нейтральные (те, у которых оценка >= 4 и <= 6).

In [4]:
def get_reviews(link):
  response = session.get(f'https://www.metacritic.com{link}/user-reviews', headers={'User-Agent': ua.random})
  soup = BeautifulSoup(response.text, "html.parser")
  reviews = soup.find_all('div', {'class': 'review pad_top1'})
  cnt = 1

  positives = []
  negatives = []
  scores = ['negative', 'positive', 'mixed']
  for review in reviews:
    if cnt > 300:
      break
    cnt += 1
    text = review.find('span', {'class': 'blurb blurb_expanded'})
    if text is None:
      text = review.find('div', {'class': 'review_body'})
      text = text.find('span')
      if text is not None:
        text = text.text
        if type(text) is not str:
          text = text.text
    f = False
    for i in range(len(scores)):
      score = review.find('div', {'class': f'metascore_w user large movie {scores[i]} indiv'})
      if score is not None and text is not None:
        f = True
        score = int(score.text)
        if score < 4:
          negatives.append(text)
        if score > 6:
          positives.append(text)
    if not f:
      score = review.find('div', {'class': 'metascore_w user large movie positive indiv perfect'})
      if score is not None and text is not None:
        positives.append(text)
  return positives, negatives

В парсинге отзывов часто бывали какие-то не до идеального конча выкачанные тексты (какие-то html-теги оставались все еще), поэтому надо было все почистить.

In [5]:
def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext

Тут происходит токенизация и лемматизация текстов (а еще для tests ниже я парсю каждый отзыв, т.е. деление на отзывы сохраняется, а для данных для "обучения" я просто создаю сразу массив лемм, чтобы дальше удобнее было работать)

In [6]:
def text_cleaning(texts, f):
  cleaned_texts = []
  for text in texts:
    sw = stopwords.words('english')
    morph = MorphAnalyzer()
    words = [w.lower() for w in word_tokenize(text) if w.isalpha()]
    filtered = [w for w in words if w not in sw]
    if f:
      cleaned_texts.extend(filtered)
    else:
      cleaned_texts.append(filtered)
  return cleaned_texts

В этой функции находим множество слов, которые встречаются в А, но не встречаются в В.

In [36]:
def sets(list_A, list_B):
  A = set(list_A)
  B = set(list_B)
  return A.difference(A.intersection(B))

In [8]:
from fake_useragent import UserAgent
from random import randint
from bs4 import BeautifulSoup
import requests
import string
import random
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
import re

In [9]:
ua = UserAgent(verify_ssl=False)
session = requests.session()

Я взяла один год, потому что иначе выходило слишком много данных и программа очень долго работала...

In [11]:
links = get_films(2008)

In [12]:
positives = []
negatives = []

In [13]:
for link in links:
  p, n = get_reviews(link)
  positives.extend(p)
  negatives.extend(n)

In [14]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [15]:
length = int(min(len(positives), len(negatives)) / 4)

Тут делаю три листа: для "обучения" (positives и negatives) и для теста (test), а так же "правильные ответы" к тесту.

In [16]:
test = []
test.extend(positives[(length * 3):(length * 4)])
y_true = [1 for i in range(len(test))]
y_true.extend([0 for j in range(len(test))])
test.extend(negatives[(length * 3):(length * 4)])

In [17]:
positives = positives[:(length * 3)]
negatives = negatives[:(length * 3)]

Проверяю, что по длинам все сошлось.

In [18]:
print(len(positives))
print(len(negatives))
print(len(test))
print(len(y_true))

186
186
124
124


Чищу оставшиеся html-теги

In [19]:
for i in range(len(positives)):
  if type(positives[i]) is not str:
    positives[i] = cleanhtml(str(positives[i]))

In [20]:
for i in range(len(negatives)):
  if type(negatives[i]) is not str:
    negatives[i] = cleanhtml(str(negatives[i]))

In [21]:
for i in range(len(test)):
  if type(test[i]) is not str:
    test[i] = cleanhtml(str(test[i]))

In [22]:
positives = text_cleaning(positives, True)
negatives = text_cleaning(negatives, True)
test = text_cleaning(test, False)

In [23]:
from collections import Counter

In [24]:
positives = Counter(positives)
negatives = Counter(negatives)

Выбираю во множествах те слова, которые встречаются >= 10 раз. Кажется, если сначала выкинуть пересение между множествами, а потом смотреть на частоты, то выйдет плохо (например, у нас может встретиться perfect в отрицательном отзыве один раз, и тогда мы его выкинем). А если возьмем сначала просто самые частотные слова и в них уже пересечение выкинем, то такой ситуации не будет.

In [34]:
pos_set = []
for i in positives.most_common(100):
  if i[1] < 10:
    break
  pos_set.append(i[0])

In [33]:
neg_set = []
for i in negatives.most_common(100):
  if i[1] < 10:
    break
  neg_set.append(i[0])

In [37]:
only_pos = sets(pos_set, neg_set)

In [39]:
only_neg = sets(neg_set, pos_set)

In [42]:
y_pred = []

Считаю сколько слов в отзыве входят в "положительное" множество и в "отрицательное". Каких больше, таким и будем считать отзыв.

In [43]:
for text in test:
  cnt_pos = 0
  cnt_neg = 0
  for w in text:
    if w in only_pos:
      cnt_pos += 1
    if w in only_neg:
      cnt_neg += 1
  y_pred.append(int(cnt_pos > cnt_neg))

In [45]:
import sklearn
from sklearn.metrics import accuracy_score

Ну и понятно, что accuracy очень не очень, но все равно неплохо - выше, чем я ожидала)

In [46]:
accuracy_score(y_true, y_pred)

0.6612903225806451

Как улучшить:

1. Понятно, увеличить корпус (но это я реализовывать не буду, слишком долго будет работать - функция токенизации и лемматизации довольно долго работает...)

2. Можно попробовать учитывать отрицания (для отрицательных отзывов скорее?)

3. Использовать метрику tf-idf. Там, помимо частотности самого слова в документах - отзывах, учитывается так же и количество документов, в которых это слово встречается. Это поможет меньше учитывать слова, которые часто встретились в малом количества отзывов (т.е. это скорее какая-то характеристика письма человека, а не характеристика фильма), типа в одном отзыве несколько раз употребилось одно и то же слово, а в других оно почти не встречалось или встречалось по одному разу, например.

4. Выкачать словарь оценочных слов (возможно, как-то попытаться полувручную это сделать?). Плохие - bad, awful, boring и т.д., хорошие - wonderful, awesome, interesting и т.д.

5. В полученных мною множествах было много мусора, который встречался часто. Например в "отрицательном" сете очень часто встречалось, например, слово vampire. Такое может быть, что жанровые (и вообще какие-то описательные штуки про фильм) встретились только в одной оценочной категории (особенно, если корпус не такой большой), поэтому можно попытаться создать словарь таких типа "жанровых" слов и выкидывать их как стоп-слова, чтобы они не засоряли картину.